# CPSC 330 - Applied Machine Learning 

## Homework 2: Decision trees and machine learning fundamentals 
### Associated lectures: [Lectures 2 and 3](https://ubc-cs.github.io/cpsc330/README.html) 

**Due date: Monday Sep 20, 2021 at 11:59pm**

In [463]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import altair as alt

plt.rcParams["font.size"] = 16

from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier

## Instructions
rubric={points:3}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330/blob/master/docs/homework_instructions.md). In particular, **see the note about not pushing downloaded data to your repo**.

You are welcome to broadly discuss questions with your classmates but your final answers must be your own. **We are not allowing group submission for this homework assignment.**  

## Introducing the data set
 
For this  assignment you'll be looking at Kaggle's [Spotify Song Attributes](https://www.kaggle.com/geomack/spotifyclassification/) dataset.
The dataset contains a number of features of songs from 2017 and a binary variable `target` that represents whether the user liked the song (encoded as 1) or not (encoded as 0). See the documentation of all the features [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/). 

This dataset is publicly available on Kaggle, and you will have to download it yourself. Follow the steps below to get the data CSV. 

1. If you do not have an account with [Kaggle](https://www.kaggle.com/), you will first need to create one (it's free).
2. Login to your account and [download](https://www.kaggle.com/geomack/spotifyclassification/download) the dataset.
3. Unzip the data file if needed, then rename it to `spotify.csv`, and move it to the same directory as this notebook.

## Exercise 1: Exploratory data analysis

#### 1(a) 
rubric={points:2}

Read in the data CSV and store it as a pandas dataframe named `spotify_df`. The first column of the .csv file should be set as the index.

In [464]:
spotify_df=pd.read_csv('data/spotify.csv', index_col=0)

#### 1(b)
rubric={points:2}

Run the following line of code to split the data. How many training and test examples do we have?

> Note: we are setting the `random_state` so that everyone has the same split on their assignments. This will make it easier for the TAs to grade.

In [465]:
df_train, df_test = train_test_split(spotify_df, test_size=0.2, random_state=321)
print("Training examples (in rows):", len(df_train), "\nTest examples (in rows):", len(df_test))

Training examples (in rows): 1613 
Test examples (in rows): 404


#### 1(c)
rubric={points:3}

- Print out the output of `describe()` **on the training split**. This will compute some summary statistics of the numeric columns.
- Which feature has the smallest range? 

> Hint: You can subtract the min value from the max value of the column to get the range.

Note that `describe` returns another DataFrame.

In [466]:
# citations for this question:
# [1] iterating over columns in DataFrame: https://thispointer.com/pandas-loop-or-iterate-over-all-or-certain-columns-of-a-dataframe/

df_train_describe = df_train.describe()
print(df_train_describe.to_string())

min_range = sys.maxsize
min_range_feature = ""
for (col_name, col_data) in df_train_describe.iteritems():  # [1]
    curr_range = col_data[7] - col_data[3]
    if curr_range < min_range:
        min_range = curr_range
        min_range_feature = col_name
print("\nMin range feature:", min_range_feature)

       acousticness  danceability   duration_ms       energy  instrumentalness          key     liveness     loudness         mode  speechiness        tempo  time_signature      valence       target
count   1613.000000   1613.000000  1.613000e+03  1613.000000       1613.000000  1613.000000  1613.000000  1613.000000  1613.000000  1613.000000  1613.000000     1613.000000  1613.000000  1613.000000
mean       0.185067      0.620076  2.462533e+05     0.681315          0.134317     5.384377     0.191317    -7.095272     0.619343     0.092119   121.310311        3.975201     0.495891     0.512089
std        0.255838      0.161152  8.056740e+04     0.206964          0.274217     3.653722     0.156071     3.678993     0.485699     0.088007    26.431574        0.247829     0.244267     0.500009
min        0.000003      0.148000  1.604200e+04     0.015600          0.000000     0.000000     0.018800   -31.082000     0.000000     0.023100    47.859000        1.000000     0.037300     0.000000
25%  

#### 1(d) 
rubric={points:5}

Let's focus on the following features:

- danceability
- tempo
- energy
- valence

For each of these features (in order), produce a histogram that shows the distribution of the feature values in the training set, **separated for positive and negative examples**. 
By "positive examples" we mean target = 1 (user liked the song, positive sentiment) and by "negative examples" we mean target = 0 (used disliked the song, negative sentiment). As an example, here is what the histogram would look like for a different feature, loudness:



<img src='loudness.png' width="400">

(You don't have to match all the details exactly, such as colour, but your histograms should look something like this, with a reasonable number of bins to see the shape of the distribution.) As shown above, there are two different histograms, one for target = 0 and one for target = 1, and they are overlaid on top of each other. The histogram above shows that extremely quiet songs tend to be disliked (more blue bars than orange on the left) and very loud songs also tend to be disliked (more blue than orange on the far right).

To adhere to the [DRY (Don't Repeat Yourself)](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself) principle, make sure you use a `for` loop for your plotting, rather than repeating the plotting code 4 times. For this to work, I used `plt.show()` at the end of your loop, which draws the figure and resets the canvas for your next plot.

Here is some code that separates out the dataset into positive and negative examples, to help you get started:

In [467]:
# citations for this question:
# [1] concatenating series into DataFrame with renamed columns: https://stackoverflow.com/questions/39047915/concat-series-onto-dataframe-with-column-name
# [2] layered altair histogram: https://altair-viz.github.io/gallery/layered_histogram.html
# [3] plotting altair charts in a for loop: https://github.com/altair-viz/altair/issues/1281#issuecomment-450783636
# [4] display one legend for each concatenated chart: https://stackoverflow.com/questions/60328943/how-to-display-two-different-legends-in-hconcat-chart-using-altair

negative_examples = df_train.query("target == 0")
positive_examples = df_train.query("target == 1")
selected_features = ["danceability", "tempo", "energy", "valence"]
negative_selected = negative_examples[selected_features]
positive_selected = positive_examples[selected_features]

charts = []
for i in range(len(negative_selected.columns)):
    curr_negative = negative_selected.iloc[:, i].reset_index(drop=True)
    curr_positive = positive_selected.iloc[:, i].reset_index(drop=True)
    feature_data_source = pd.concat([curr_negative.rename("0"), curr_positive.rename("1")], axis=1)   # [1]
    
    altair_chart = alt.Chart(feature_data_source).transform_fold(                                     # [2]
        ["0", "1"],
        as_=['Feature target class', curr_negative.name]
    ).mark_area(
        opacity=0.5,
        interpolate='step'
    ).properties(
        title="Histogram of %s by target class" % curr_negative.name
    ).encode(
        alt.X(curr_negative.name + ':Q', bin=alt.Bin(maxbins=50)),
        alt.Y('count()', stack=None),
        alt.Color('Feature target class:N')
    )
    charts.append(altair_chart)                                                                       # [3]

charts_res = alt.vconcat(*charts).resolve_scale(color='independent')                                               # [3][4]
charts_res
# charts do not show on gradescope for some reason although they show in my jupyter notebook environment :( please see my attached screenshots
# I followed section 20 of this module: https://ml-learn.mds.ubc.ca/en/module3

alt.VConcatChart(...)

#### 1(e)
rubric={points:4}

Let's say you had to make a decision stump (decision tree with depth 1), _by hand_, to predict the target class. Just from looking at the plots above, describe a reasonable split (feature name and threshold) and what class you would predict in the two cases. For example, in the loudness histogram provided earlier on, it seems that very large values of loudness are generally disliked (more blue on the right side of the histogram), so you might answer something like this: "A reasonable split would be to predict 0 if loudness > -5 (and predict 1 otherwise)."

A reasonable split would be to predict 1 if danceability is > 0.78

#### 1(f)
rubric={points:2}

Let's say that, for a particular feature, the histograms of that feature are identical for the two target classes. Does that mean the feature is not useful for predicting the target class?



When the histogram of that feature is identical for the two target classes, it means that the number of records corresponding to each feature value is split evenly.
It does not mean that the feature is not useful - it just means that if a model is trained with that feature, there is a 50% chance of each target class being observed given any value that falls within the observed values.

#### 1(g) 
rubric={points:2}

Note that the dataset includes two free text features labeled `song_title` and `artist`:

In [468]:
df_train[["song_title", "artist"]].head()

,song_title,artist
260,WTF (Where They From) [feat. Pharrell Williams],Missy Elliott
1286,"10,000 Reasons (Bless the Lord) [Radio Version]",Matt Redman
1344,American Dream,Chelsea Grin
1197,Feel This Moment,Pitbull
119,Trap Queen,Fetty Wap


- Do you think these features could be useful in predicting whether the user liked the song or not? 
- Would there be any difficulty in using them in your model?   

They would be useful in predicting whether or not the user liked the song, because people may like songs written by certain artists, or songs that contain a special word in the title.

<br><br>

## Exercise 2: Using sklearn to build a decision tree classifier

#### 2(a) 
rubric={points:2}

- Create `X_train` and `y_train` and `X_test` and `y_test` from `df_train` and `df_test` above. Skip the `song_title` and `artist` features for now. 
- Fit a `DecisionTreeClassifier` on the train set.

In [469]:
X_train = df_train.drop(["target", "song_title", "artist"], axis=1)
X_test = df_test.drop(["target", "song_title", "artist"], axis=1)
y_train = df_train["target"]
y_test = df_test["target"]
model = DecisionTreeClassifier(random_state=321)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=321)

#### 2(b)
rubric={points:2}

Use the `predict` method to predict the class of the first example in your `X_train`. Is the prediction correct? That is, does it match with the corresponding class in `y_train`?  

> Hint: you can grab the first example with `X_train.iloc[[0]]`.

In [470]:
predicted = model.predict(X_train.iloc[[0]])
display_predicted = pd.concat([df_train.drop(["song_title", "artist"], axis=1).iloc[[0]].reset_index(drop=True), pd.Series(predicted, name='predicted')], axis=1)
print('The prediction is correct; it matches with the corresponding y_train (as the target column within df_train). See the last column:')
display_predicted

The prediction is correct; it matches with the corresponding y_train (as the target column within df_train). See the last column:


,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,predicted
0,0.0181,0.932,192773,0.819,0.000007,8,0.0577,-3.484,0,0.203,119.941,4.0,0.552,1,1


#### 2(c) 
rubric={points:2}

Use the `cross_val_score` function on your training set to compute the 10-fold cross-validation accuracy of your tree. 

In [471]:
cv_score = cross_val_score(model, X_train, y_train, cv=10)
cv_score

array([0.7037037 , 0.62962963, 0.66049383, 0.71428571, 0.7515528 ,
       0.61490683, 0.70186335, 0.74534161, 0.63354037, 0.68944099])

#### 2(d)
rubric={points:2}

The above is useful, but we would like to see the training accuracy as well. 

- Compute the 10-fold cross-validation again but this time using the `cross_validate` function with `return_train_score=True`. 
- Print out both the cross-validation score and the training score.
- Is your cross-validation score exactly the same as what you got in the previous part? Very briefly discuss.

In [472]:
scores = cross_validate(model, X_train, y_train, cv=10, return_train_score=True)
scores_df = pd.DataFrame(scores)
scores_df_mean = scores_df.mean()
training_score = model.score(X_train, y_train)
print("The cross-validation scores for each fold:\n", scores_df)
print("\nThe training score: %0.6f" % training_score)
print("\n=================")

scores_df_mean_test_score = scores_df_mean.iloc[2]
cv_score_mean = cv_score.mean()
print("\nThe mean cross-validation scores in this question %0.6f:" % scores_df_mean_test_score)
print("\nThe mean cross-validation score from 2c: %0.6f" % cv_score_mean)
print("\nThe cross-validation scores (represented by test_score) in this question is the same as the cross-validation score in the previous question.")
print('Both cross_validate and cross_val_score are using the same random_state=321, which means that the scores are computed from the same splits (example rows).')

The cross-validation scores for each fold:
    fit_time  score_time  test_score  train_score
0  0.027605    0.003588    0.703704     0.999311
1  0.030504    0.003922    0.629630     0.998622
2  0.028457    0.002266    0.660494     0.999311
3  0.022920    0.001636    0.714286     0.998623
4  0.016269    0.002088    0.751553     0.998623
5  0.016572    0.001564    0.614907     0.998623
6  0.015616    0.002036    0.701863     0.998623
7  0.015579    0.002294    0.745342     0.999311
8  0.017138    0.001534    0.633540     0.999311
9  0.014733    0.001499    0.689441     0.998623

The training score: 0.998760


The mean cross-validation scores in this question 0.684476:

The mean cross-validation score from 2c: 0.684476

The cross-validation scores (represented by test_score) in this question is the same as the cross-validation score in the previous question.
Both cross_validate and cross_val_score are using the same random_state=321, which means that the scores are computed from the same 

#### 2(e)
rubric={points:1}

Do you see a significant difference between the training score and the cross-validation score? Briefly discuss.

In [473]:
print('Training score calculated in 2(d): %f' % training_score)
print('Average of 10-fold cross-validation scores calculated in 2(d): %f3' % scores_df_mean_test_score)
print('Compared to the training score calculated in 2(d), the cross-validation score is quite far away.')
print("The mean cross-validation score from all the splits is around 0.3 points lower than the training score - this suggests that we're likely overfitting.")

Training score calculated in 2(d): 0.998760
Average of 10-fold cross-validation scores calculated in 2(d): 0.6844763
Compared to the training score calculated in 2(d), the cross-validation score is quite far away.
The mean cross-validation score from all the splits is around 0.3 points lower than the training score - this suggests that we're likely overfitting.


#### 2(f)
rubric={points:1}

Inspect the 10 sub-scores from the 10 folds of cross-validation. How does this inform the trustworthiness of your cross validation score?

In [474]:
print('The 10 sub-scores from the 10 folds of cross-validation (mean = %f) is same as the cross-validation score (mean of cross-validation scores: %f) - this means that the dataset is sufficiently large and insensitive to splitting behaviour, making the cross-validation score trustworthy.' % (scores_df_mean_test_score, cv_score_mean))
scores_df

The 10 sub-scores from the 10 folds of cross-validation (mean = 0.684476) is same as the cross-validation score (mean of cross-validation scores: 0.684476) - this means that the dataset is sufficiently large and insensitive to splitting behaviour, making the cross-validation score trustworthy.


,fit_time,score_time,test_score,train_score
0,0.027605,0.003588,0.703704,0.999311
1,0.030504,0.003922,0.629630,0.998622
2,0.028457,0.002266,0.660494,0.999311
3,0.022920,0.001636,0.714286,0.998623
4,0.016269,0.002088,0.751553,0.998623
5,0.016572,0.001564,0.614907,0.998623
6,0.015616,0.002036,0.701863,0.998623
7,0.015579,0.002294,0.745342,0.999311
8,0.017138,0.001534,0.633540,0.999311
9,0.014733,0.001499,0.689441,0.998623


## Exercise 3: Hyperparameters 
rubric={points:10}

In this exercise, you'll experiment with the `max_depth` hyperparameter of the decision tree classifier. See the [`DecisionTreeClassifier` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for more details.

- Explore the `max_depth` hyperparameter. Run 10-fold cross-validation for trees with different values of `max_depth` (at least 10 different values in the range 1 to 25).
- For each `max_depth`, get both the train accuracy and the cross-validation accuracy.
- Make a plot with `max_depth` on the *x*-axis and the train and cross-validation scores on the *y*-axis. That is, your plot should have two curves, one for train and one for cross-validation. Include a legend to specify which is which.
- Discuss how changing the `max_depth` hyperparameter affects the training and cross-validation accuracy. From these results, what depth would you pick as the optimal depth? 
- Do you think that the depth you chose would generalize to other "spotify" datasets (i.e., data on other spotify users)?

> Note: generally speaking (for all assignments) you are welcome to copy/paste code directly from the lecture notes, though I ask that you add a small citation (e.g. "Adapted from lecture 2") if you do so.

In [475]:
# [citation] Adapted from Introduction to Machine Learning, Module 3, section 20: https://ml-learn.mds.ubc.ca/en/module3

res = {'depth': [], 'mean_train_score': [], 'mean_cv_score': []}
max_cv_score = -sys.maxsize
max_depth = 1;
for i in range(1, 25):
    model = DecisionTreeClassifier(max_depth=i, random_state=321)
    scores = cross_validate(model, X_train, y_train, cv=10, return_train_score=True)
    res['depth'].append(i)
    res['mean_train_score'].append(scores['train_score'].mean())
    mean_cv_score = scores['test_score'].mean()
    res['mean_cv_score'].append(mean_cv_score)
    if (mean_cv_score > max_cv_score):
        max_cv_score = mean_cv_score
        max_depth = i

res_df = pd.DataFrame(res).melt(id_vars=['depth'],
                                value_vars=['mean_train_score',
                                            'mean_cv_score'],
                                var_name='split',
                                value_name='score')

chart = alt.Chart(res_df).mark_line().properties(
            title='Max Depth of DecisionTreeClassifier vs. Train and Cross-Validation Accuracy'
        ).encode(
            alt.X('depth:Q', axis=alt.Axis(title='Max Tree Depth')),
            alt.Y('score:Q', axis=alt.Axis(title='Train and Cross-Validation Accuracy'),
                            scale=alt.Scale(domain=[.50, 1.00])),
            alt.Color('split:N', scale=alt.Scale(domain=['mean_train_score',
                                                         'mean_cv_score'],
                                                 range=['purple', 'gold'])))
print('How changing the max_depth hyperparameter affects the training and cross-validation accuracy:')
print('Increasing the max_depth hyperparameter increases training accuracy, but once the max_depth of the decision tree gets too deep, increasing the depth rather reduces the cross-validation accuracy.')
print('For example, as the max_depth increases beyond %0.1f, the cross-validation accuracy starts to decrease from around %0.4f to 0.71.' % (max_depth, max_cv_score))
print('This is not a significant decrease, but it means that when the depth of the tree increases, although the training accuracy increases, the model starts to pick up on more quirks of the training dataset.')
print('Once the decision tree\'s decisions get too complex, we risk overfitting because the patterns we picked up from the training sets are too specific to be used for the validation sets, increasing the validation error.')

print('\nOptimal depth: %0.1f' % max_depth)

print('\nWould this optimal depth generalize to other spotify datasets: ')
print('If this model is generalized on other datasets significantly larger in size, our model may not generalize well - there are millions of spotify users, and the size of our training data is only in the thousands (n = %f). \n' % len(X_train))

chart

How changing the max_depth hyperparameter affects the training and cross-validation accuracy:
Increasing the max_depth hyperparameter increases training accuracy, but once the max_depth of the decision tree gets too deep, increasing the depth rather reduces the cross-validation accuracy.
For example, as the max_depth increases beyond 4.0, the cross-validation accuracy starts to decrease from around 0.7328 to 0.71.
This is not a significant decrease, but it means that when the depth of the tree increases, although the training accuracy increases, the model starts to pick up on more quirks of the training dataset.
Once the decision tree's decisions get too complex, we risk overfitting because the patterns we picked up from the training sets are too specific to be used for the validation sets, increasing the validation error.

Optimal depth: 4.0

Would this optimal depth generalize to other spotify datasets: 
If this model is generalized on other datasets significantly larger in size, our

alt.Chart(...)

## Exercise 4: Test set
rubric={points:4}

Remember the test set you created way back at the beginning of this assignment? Let's use it now to see if our cross-validation score from the previous exercise is trustworthy. 

- Select your favorite `max_depth` from the previous part.
- Train a decision tree classifier using that `max_depth` on the _entire training set_.
- Compute and display the test score. 
- How does it compare to the cross-validation score from the previous exercise? Briefly discuss. 

In [476]:
model = DecisionTreeClassifier(max_depth=max_depth, random_state=321)
model.fit(X_train, y_train)
test_score = model.score(X_test, y_test)
print("The test score of our model with max_depth=%0.1f: %0.6f" % (max_depth, test_score))
print('\nThe computed test_score (%0.6f) is comparable with the cross-validation score (%0.6f) obtained with max_depth=%0.1f' % (test_score, max_cv_score, max_depth))
print('\nSince the test set likely has some \'quirks\' not found within our validation and training sets, it\'s expected that our model won\'t fit all the random patterns found in the test set.')
print('As a result, we expect the test accuracy to be slightly lower than the validation score (and the test error, 1-test_score, to be higher than the validation error) - this illustrates the fundamental tradeoff of supervised learning.')

The test score of our model with max_depth=4.0: 0.693069

The computed test_score (0.693069) is comparable with the cross-validation score (0.732781) obtained with max_depth=4.0

Since the test set likely has some 'quirks' not found within our validation and training sets, it's expected that our model won't fit all the random patterns found in the test set.
As a result, we expect the test accuracy to be slightly lower than the validation score (and the test error, 1-test_score, to be higher than the validation error) - this illustrates the fundamental tradeoff of supervised learning.


## Exercise 5: Conceptual questions
rubric={points:3}

Consider the dataset below, which has $6$ examples and $2$ features:

$$ X = \begin{bmatrix}5 & 2\\4 & 3\\  2 & 2\\ 10 & 10\\ 9 & -1\\ 9& 9\end{bmatrix}, \quad y = \begin{bmatrix}-1\\-1\\+1\\+1\\+1\\+1\end{bmatrix}.$$

1. Say we fit a decision stump (depth 1 decision tree) and the first split is on the first feature (left column) being less than 5.5. What would we predict in the "true" and "false" cases here?
2. What training accuracy would the above stump get on this data set?
3. Can we obtain 100% accuracy with a single decision stump in this particular example?

In [477]:
# citations
# [1] concatenate dataframes: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# [2] update values in pandas dataframe: https://www.kite.com/python/answers/how-to-update-a-value-in-each-row-of-a-pandas-dataframe-in-python#:~:text=Update%20elements%20of%20a%20column,the%20column%20to%20be%20changed.
df_5 = pd.DataFrame({'X0': [5,4,2,10,9,9], 'X1': [2,3,2,10,-1,9], 'target': [-1,-1,1,1,1,1]})
X_train = df_5.drop(['target', 'X1'], axis=1)
y_train = df_5['target']
df_w_predicted = pd.DataFrame({"predicted": [0,0,0,0,0,0]})
df_w_predicted = pd.concat([X_train, df_w_predicted], axis=1, join='inner')
for i in range(len(X_train)):
    predicted = 1 if df_w_predicted.at[i, "X0"] < 5.5 else -1
    df_w_predicted.at[i, "predicted"] = predicted
df_w_predicted

num_correct = 0;
for i in range(len(X_train)):
    if (df_w_predicted.at[i, "predicted"] == y_train[[i]].values[0]):
        num_correct+=1
training_accuracy = (num_correct / len(X_train))

print('1) What would we predict in the "true" and "false" cases?\n')
print(df_w_predicted)

print('\n2) Training accuracy of above stump: ' + '%0.4f' % training_accuracy)
print('\n3) Can we obtain 100% accuracy with a single decision stump in this particular example? In this case, we can make a decision stump with the condition such that if the first feature is greater than 2 AND less than 6, we return false')

1) What would we predict in the "true" and "false" cases?

   X0  predicted
0   5          1
1   4          1
2   2          1
3  10         -1
4   9         -1
5   9         -1

2) Training accuracy of above stump: 0.1667

3) Can we obtain 100% accuracy with a single decision stump in this particular example? In this case, we can make a decision stump with the condition such that if the first feature is greater than 2 AND less than 6, we return false


## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 